# Read the question Wrong so made an memory RAG model for chat purpose

In [1]:
from langchain_community.document_loaders import PyPDFLoader

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

Making Embeddings

In [2]:
# Load Documents using PyPDF LOader
loader = PyPDFLoader("Big Mac Index.pdf") # change document Here
pages = loader.load()

In [3]:
# split documents
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

In [4]:
embeddings=OpenAIEmbeddings() # open ai emebeddings
db = FAISS.from_documents(docs,embeddings ) # creating Faiss Vector store


In [5]:
db.save_local("faiss_index") # saving Faiss index in local


In [6]:
embeddings=OpenAIEmbeddings() # open ai emebeddings

db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)# loading saved Faiss index in local


In [7]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# retriver added 
retriever = db.as_retriever()


# RAG with Memory

In [8]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

### Contextualize question 
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# There are multiple ways to invoke history this way we are summarizing the history to make a new question which will contain info



In [9]:

### Answer question ###
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)



In [10]:
from langchain_core.messages import HumanMessage
chat_history = []

# save chat history

# Question answer about the Document

In [11]:
question = "which is the cheapest among the costliest one"


In [12]:
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

ai_msg_1["answer"]

'Among the most expensive places to buy a Big Mac in July 2023, the cheapest is Sweden at $5.74 (60.27 SEK).'

In [13]:
question = "what was my last question"


In [14]:
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

ai_msg_1["answer"]

'Your last question was about which of the most expensive places to buy a Big Mac was the cheapest in July 2023.'